<a href="https://colab.research.google.com/github/zhpinkman/hugging-face-projects/blob/master/MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.5 MB 5.1 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
     |████████████████████████████████| 6.8 MB 34.5 MB/s 
     |████████████████████████████████| 895 kB 41.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 311 kB 5.0 MB/s 
     |████████████████████████████████| 243 kB 54.6 MB/s 
     |████████████████████████████████| 133 kB 66.6 MB/s 
     |████████████████████████████████| 1.1 MB 57.4 MB/s 
     |████████████████████████████████| 271 kB 70.2 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 75.6 MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

checkpoint = 'distilbert-base-uncased'


tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForMaskedLM.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

In [3]:
prompt = 'That was a great [MASK].'

In [5]:
import numpy as np
import torch as th
inputs = tokenizer(prompt, return_tensors='pt')

logits = model(**inputs).logits

masked_word_index = th.where(inputs.input_ids == tokenizer.mask_token_id)[1]

masked_word_logits = logits[0, masked_word_index,:].squeeze()

top_words_ids = th.topk(masked_word_logits, 5).indices.tolist()

for word_id in top_words_ids:
  print(prompt.replace(tokenizer.mask_token, tokenizer.convert_ids_to_tokens(word_id)))

That was a great deal.
That was a great idea.
That was a great success.
That was a great accomplishment.
That was a great mistake.


In [6]:
from datasets import load_dataset

dataset = load_dataset('imdb')

dataset

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [7]:
sample = dataset['train'].select(np.random.choice(dataset['train'].num_rows, 2))[:2]
sample

{'label': [1, 0],
 'text': ['The plot intellect is about as light as feather down. But the advantage here is the boy and girl classic refusal we have become accustomed to in "The Gay Divorcee" and "Top Hat" is now absent. Instead of the typical accidental acquaintance, the dancing duo are the former lovers Bake Baker and Sherry Martin, who are still in love since their dancing days.<br /><br />Of course, being a 30s musical, there\'s the problems of misunderstood romance, classy courtship and the slight irritation of a sabotaged audition with bicarbonate soda has costing Ginger something rather special. And then in the grand tradition of dwindling finances, there\'s nothing better for Hollywood\'s best entertainers than put on a show.<br /><br />Delightful numbers from Irving Berlin are sprinkled throughout the show. Top hats and evening dresses are saved right until the end, which remains a refreshing change. Fred and Ginger are out again to charm the world...and charm the navy. Every

In [12]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

tokenized_dataset = dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_dataset

  0%|          | 0/25 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-f760cc293630cb85.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-0ad9d2e44fe200c8.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [13]:
model.config.max_position_embeddings

512

In [14]:
chunk_size = 128

In [15]:
def group_all_texts(examples):
  examples = {
      key: sum(value, [])
      for key, value in examples.items()
  }

  total_length = (len(examples['input_ids']) // chunk_size) * chunk_size

  examples = {
      key: [
            values[i: i + chunk_size]
            for i in range(0, total_length, chunk_size)
      ]
      for key, values in examples.items()
  }
  examples['labels'] = examples['input_ids'].copy()
  return examples
tokenized_dataset = tokenized_dataset.map(group_all_texts, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [16]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

In [19]:
print(tokenizer.decode(tokenized_dataset['train'][0]['input_ids']))
print(tokenizer.decode(tokenized_dataset['train'][0]['labels']))

[CLS] i rented i am curious - yellow from my video store because of all the controversy that surrounded it when it was first released in 1967. i also heard that at first it was seized by u. s. customs if it ever tried to enter this country, therefore being a fan of films considered " controversial " i really had to see this for myself. < br / > < br / > the plot is centered around a young swedish drama student named lena who wants to learn everything she can about life. in particular she wants to focus her attentions to making some sort of documentary on what the average swede thought about certain political issues such
[CLS] i rented i am curious - yellow from my video store because of all the controversy that surrounded it when it was first released in 1967. i also heard that at first it was seized by u. s. customs if it ever tried to enter this country, therefore being a fan of films considered " controversial " i really had to see this for myself. < br / > < br / > the plot is cent

In [20]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [25]:
samples = [tokenized_dataset["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] i rented i am curious - yellow from my video store [MASK] of all [MASK] controversy that surrounded it when it was first [MASK] in 1967. [MASK] [unused238] heard that at first it was seized by u. s. customs if it ever tried to enter this country shaman therefore being a fan of films considered " controversial " i really had [MASK] see this for myself. [MASK] br / > [MASK] br / > the plot is centered around [MASK] young swedish drama student [MASK] [MASK] who wants [MASK] learn everything she can about life. in particular she wants to focus her [MASK]s to making [MASK] sort of documentary on what the average swede thought about certain [MASK] [MASK] such'

'>>> as [MASK] vietnam war [MASK] race issues in the united states [MASK] in between asking politicians and ordinary [MASK]izens [MASK] stockholm about their opinions on [MASK], she [MASK] sex with her drama teacher, classmates [MASK] and married men. < br / > < br / > what kills [MASK] about i am [MASK] - yellow [MASK] th

In [26]:
small_dataset = tokenized_dataset['train'].train_test_split(
  test_size=.1, seed=41
)
small_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 55161
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 6130
    })
})

In [29]:
from transformers import TrainingArguments

batch_size = 64

logging_steps = len(small_dataset["train"]) // batch_size

args = TrainingArguments(
    output_dir=f"finetuned-imdb",
    overwrite_output_dir=True,
    evaluation_strategy="steps",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    logging_steps=logging_steps,
)


In [30]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = args, 
    train_dataset=small_dataset["train"],
    eval_dataset=small_dataset["test"],
    data_collator=data_collator,
)

Using amp half precision backend


In [31]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 55161
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2586


Step,Training Loss,Validation Loss
861,2.553100,2.342328
1722,2.438000,2.318035
2583,2.404300,2.285955


Saving model checkpoint to finetuned-imdb/checkpoint-500
Configuration saved in finetuned-imdb/checkpoint-500/config.json
Model weights saved in finetuned-imdb/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids.
***** Running Evaluation *****
  Num examples = 6130
  Batch size = 64
Saving model checkpoint to finetuned-imdb/checkpoint-1000
Configuration saved in finetuned-imdb/checkpoint-1000/config.json
Model weights saved in finetuned-imdb/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to finetuned-imdb/checkpoint-1500
Configuration saved in finetuned-imdb/checkpoint-1500/config.json
Model weights saved in finetuned-imdb/checkpoint-1500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids.
***** Running Evaluation *****
  Num examples 

TrainOutput(global_step=2586, training_loss=2.4650572804821986, metrics={'train_runtime': 1009.4166, 'train_samples_per_second': 163.939, 'train_steps_per_second': 2.562, 'total_flos': 5484154448558592.0, 'train_loss': 2.4650572804821986, 'epoch': 3.0})

In [35]:
device = th.device('cuda') if th.cuda.is_available() else th.device('cpu')

In [36]:
device

device(type='cuda')

In [40]:
prompt = 'This is a great [MASK].'

inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

logits = model(**inputs).logits

masked_word_index = th.where(inputs.input_ids == tokenizer.mask_token_id)[1]

masked_word_logits = logits[0, masked_word_index,:].squeeze()

top_words_ids = th.topk(masked_word_logits, 5).indices.tolist()

for word_id in top_words_ids:
  print(prompt.replace(tokenizer.mask_token, tokenizer.convert_ids_to_tokens(word_id)))

This is a great film.
This is a great movie.
This is a great idea.
This is a great show.
This is a great comedy.
